In [2]:
from _setup import _setup_notebook

_setup_notebook()

import pandas as pd

pd.set_option("display.max_columns", 50)

# Hands-on opgave

## Beskrivelse
Udvikl en model, der prædikterer den helårlige sandsynlighed for en eller flere skader på en indbrudsforsikring.

Modellen skal *klassificere* kunder med indbrudsforsikring bedst muligt.

## Data 

Modellen skal udvikles med afsæt i data for et fiktivt forsikringsselskab. 

Datasættet indeholder oplysninger om skader fra 2015-2019.

Disse data lå til grund for bacheloropgaver i Skadesforsikringsmatematik i 2020 - har nogen af jer lavet den?

En oversigt over variablene i datasættet med tilhørende (korte) forklaringer er tabelleret i `data/covariates.pdf`.

### Første kig på data

In [4]:
df = pd.read_parquet("../data/train.parquet")
df.head(n=6)

,idnummer,aar,Kat,expo,skadesantal,alder,forsum,zone,anc,afst_brand,afst_politi,anthund,boligareal,selvrisk,segment,ant_vaadrum,betfrekvens,byg_anvend_kode,kon,lejlighed,opfaar,parcel,tagtype,varmeinst,geo,skadesudgift
0,280135,2015,Indbrud,1.0,0,63,949000,1,7,7.279781,0.890548,2,108.0,0,3,3,1,120,M,0,2012,1,5,5,landsby,0.0
1,242129,2015,Indbrud,1.0,0,61,759000,8,6,2.104601,2.059088,0,175.0,0,3,5,3,120,K,0,1864,1,3,2,land,0.0
2,253578,2015,Indbrud,1.0,0,27,190000,3,1,1.654300,1.055982,0,58.0,1000,3,3,3,140,M,1,1947,0,5,1,metropol,0.0
3,195235,2015,Indbrud,1.0,0,51,1139000,2,19,0.633819,0.925148,0,157.0,0,3,4,3,120,K,0,1974,1,4,1,landsby,0.0
4,232655,2015,Indbrud,1.0,0,<NA>,949000,8,32,3.310923,4.494339,0,67.0,1000,3,3,1,140,NaN,1,1979,0,4,1,by,0.0
5,94021,2015,Indbrud,1.0,0,81,949000,2,28,14.207866,0.599468,0,85.0,0,3,3,3,130,M,0,1991,1,4,2,land,0.0


#### Noter

- Rækkerne er unikke på kombinationen af `idnummer` og `aar`.
- Der indgår kun observationer, hvor `expo == 1`.
- `skadesantal` er antallet af opgjorte skader for det pågældende år.
- Der indgår kun observationer, hvor `Kat == 'Indbrud'`.

## Formulering af ML-opgave
Vi definerer vores target $y$ som en binær variabel `skadesantal > 0`. Dermed står vi med et binært klassifikationsproblem.

De øvrige variable i datasættet kan potentielt anvendes som features, $X$, i vores model. 

Vores model skal kunne estimere sandsynligheden $P(y_{i}=1|X_i)$ for enhver observation $i$.

Vi vedtager, at modellens prædiktive performance skal maksimere performance-metrikken [*ROC AUC*](05_roc_auc.ipynb) (som vi vender tilbage til).

## Data splitting
Før vi begynder selve udviklingen af vores modeller, vil vi splitte vores datasæt i tre sæt: *træning*, *validering* og *test*.

- *træning*: bruges til at træne modeller.
- *validering*: bruges til at tune modellens parametre og vurdere modellens prædiktive performance på i forbindelse med træning.
- *test*: bruges til at evaluere modellens endelige performance, når man har valgt sin endelige model og *er __helt færdig__ med at modellere*. 

### Motivation for data splitting

- at udvikle robuste ML-modeller, der generaliserer bedst muligt til nye, usete data
- at sikre unbiased estimation af modellens performance

Som standard samples de tre datasæt tilfældigt blandt observationerne i datasættet, f.eks. i et 70/15/15-split.

![Data Splitting](img/datasplit.png)

I vores case er *observationerne for 2019 på forhånd udtaget til et test-sæt*.

Se også [Train, Test and Validation Sets](https://mlu-explain.github.io/train-test-validation/).

## [LAB] 🔬 

&#128073; Indledende data-analyse: `lab_01_eda.ipynb`